In [1]:
# dependencies for google colab
! pip install langchain_google_genai
! pip install python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


In [31]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
import re

from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
from langchain.prompts import ChatPromptTemplate

from diffusers import DiffusionPipeline


In [3]:

# from transformers import Blip2Processor, Blip2ForConditionalGeneration
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# model = Blip2ForConditionalGeneration.from_pretrained(
#     "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
# )
# model.to(device)
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

# generated_ids = model.generate(**inputs)
# generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
# print(generated_text)

In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")


# loading stable diffusion
pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5")
pipe = pipe.to('cuda')




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [5]:
img_url = 'https://media.istockphoto.com/id/689616366/vector/kids-doodles-hand-drawing-unicorn-running-on-rainbow.jpg?s=1024x1024&w=is&k=20&c=lRb6FO8AyraMpQ8dfyuYcT51wCQJMKkUWbK8KWTw-is='
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

In [6]:
# # conditional image captioning
# text = "a photography of"
# inputs = processor(raw_image, text, return_tensors="pt").to("cuda")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a drawing of a unicorn on a rainbow with a sun in the background


In [7]:
model_prompt=processor.decode(out[0], skip_special_tokens=True)

In [8]:
model_prompt

'a drawing of a unicorn on a rainbow with a sun in the background'

In [9]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"

# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [10]:
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [20]:
requirement = input("Please input your requirement for the story: ")

Please input your requirement for the story: exciting, for an 8 years old kid


In [23]:
query =f' Write a short story based on the content of {model_prompt} and that fits the following requirements: {requirement}'

In [30]:
result = llm.invoke(query)
print(result)

content='Sparklehoof, a unicorn with a mane like spun moonlight and a horn that shimmered like a thousand tiny stars, wasn\'t your average unicorn.  She hated boring clouds!  She longed for adventure, and today, adventure found her.\n\nSparklehoof was grazing in a field of rainbow-colored flowers when the ground beneath her feet began to jiggle.  Then, *WHOOSH!*  A giant, shimmering rainbow arched across the sky, landing right in front of her!  It wasn\'t just any rainbow; it was bouncy!  It felt like a giant, sugary slide.\n\n"Whee!" squealed Sparklehoof, and she galloped onto the rainbow, her hooves making happy *boing* sounds.  The rainbow arched higher and higher, carrying her towards a sun that looked like a giant, golden lollipop.  \n\nAs she soared, she saw amazing things!  Tiny, giggling clouds chased each other, fluffy sheep with rainbow wool leaped over waterfalls of sparkling water, and mischievous pixies swung from giant, glittering raindrops.\n\nSuddenly, a grumpy cloud, a

In [34]:
paragraphs=re.split(r'\n\s*\n', result.content)

In [40]:
paragraphs

["Sparklehoof, a unicorn with a mane like spun moonlight and a horn that shimmered like a thousand tiny stars, wasn't your average unicorn.  She hated boring clouds!  She longed for adventure, and today, adventure found her.",
 "Sparklehoof was grazing in a field of rainbow-colored flowers when the ground beneath her feet began to jiggle.  Then, *WHOOSH!*  A giant, shimmering rainbow arched across the sky, landing right in front of her!  It wasn't just any rainbow; it was bouncy!  It felt like a giant, sugary slide.",
 '"Whee!" squealed Sparklehoof, and she galloped onto the rainbow, her hooves making happy *boing* sounds.  The rainbow arched higher and higher, carrying her towards a sun that looked like a giant, golden lollipop.  ',
 'As she soared, she saw amazing things!  Tiny, giggling clouds chased each other, fluffy sheep with rainbow wool leaped over waterfalls of sparkling water, and mischievous pixies swung from giant, glittering raindrops.',
 'Suddenly, a grumpy cloud, as gre

In [ ]:
Hugging_face_token=os.getenv('hugging_face_token')

In [ ]:
! huggingface-cli login --token $Hugging_face_token

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5")
pipe = pipe.to('cuda')

In [ ]:
# prompt = result["text"]
prompt= 'one can of coca cola, on a wooden bench in a park, 8k'
image = pipe(prompt).images[0]

In [ ]:
# ouputs:
generated_story=result["text"]
generated_image=image
generated_image